# Load WOD18 data

Loaded vars are: temp, sal, o2  
Derived vars are: sst, temp100, tcd, sss, o2, o2100, po2, po2100, thd

#### Load temp, sal, o2 from WOD18 nc files

In [6]:
ncfile = dpath + 'WOD18/temp_195501-201807_5deg.nc'
temp = xr.open_dataset(ncfile) # degC
temp = temp['Temp']; temp.name = 'temp'

ncfile = dpath + 'WOD18/sal_195501-201807_5deg.nc'
sal = xr.open_dataset(ncfile) # psu
sal = sal['Sal']; sal.name = 'sal'

ncfile = dpath + 'WOD18/o2_195501-201807_5deg.nc'
o2 = xr.open_dataset(ncfile) # umol/kg
o2 = o2['O2']; o2.name = 'o2'

#### Correct the time axis

In [7]:
# - Time goes from Jan 1955 - Jul 2018 --> 763 months
begdatewod = datetime.date(year = 1955, month = 1, day = 1)
enddatewod = datetime.date(year = 2018, month = 7, day = 1)
timemowod = pd.date_range(begdatewod, enddatewod, freq='MS')

In [8]:
o2['time'] = timemowod
sal['time'] = timemowod
temp['time'] = timemowod

#### Change lon from -180-180 to 0-360

In [9]:
temp['lon'] = np.mod(temp['lon'], 360)
temp = temp.reindex({'lon': np.sort(temp['lon'])}) # or: temp = temp.roll(lon=36)
o2['lon'] = np.mod(o2['lon'], 360)
o2 = o2.reindex({'lon': np.sort(o2['lon'])}) # or: o2 = o2.roll(lon=36)
sal['lon'] = np.mod(sal['lon'], 360)
sal = sal.reindex({'lon': np.sort(sal['lon'])}) # or: sal = sal.roll(lon=36)

#### Calculate derived variables

In [10]:
# - Grab sst, temp100, sss, o2100
sst = temp.isel(depth=0); sst.name = 'sst'
sst.attrs['long_name'] = 'sea surface temperature'

temp100 = temp.sel(depth=100); temp100.name = 'temp100'
temp100.attrs['long_name'] = 'temperature at 100m'

sss = sal.isel(depth=0); sss.name = 'sss'
sss.attrs['long_name'] = 'sea surface salinity'

o2100 = o2.sel(depth=100); o2100.name = 'o2100'
o2100.attrs['long_name'] = 'o2 at 100m'

In [13]:
# - Compute po2; grab po2100
loadpo2 = 1
if loadpo2==0:
    po2 = calc_po2(o2,temp,sal,o2.depth)
    po2.name = 'po2'
    po2.attrs['long_name'] = 'oxygen partial pressure'
    po2.attrs['units'] = 'kPa'
    po2.to_netcdf(dpath + 'WOD18/po2_195501-201807_5deg.nc')
elif loadpo2==1:
    ncfile = dpath + 'WOD18/po2_195501-201807_5deg.nc' 
    po2 = xr.open_dataset(ncfile) # kPa
    po2 = po2['po2']
po2100 = po2.sel(depth=100); po2100.name = 'po2100'
po2100.attrs['long_name'] = 'po2 at 100m'
po2100.attrs['units'] = 'kPa'

In [14]:
# - Compute tuna hypoxic depth (THD)
loadthd = 1
if loadthd==0:
    thd = xr.full_like(o2100,np.nan)
    thd.name = 'thd'; thd.attrs['units'] = 'm';
    thd.attrs['long_name'] = 'tuna hypoxic depth (15kPa)'
    for ilon in range(thd.lon.size):
        print(ilon)
        for ilat in range(thd.lat.size):
            for itime in range(thd.time.size):
                po2profnow = po2.isel(lat=ilat,lon=ilon,time=itime)
                po2profnow = po2profnow[~np.isnan(po2profnow)]
                if len(po2profnow)>0: # get rid of case in which po2profnow is all nans
                    if (po2profnow.min()<=thp) & (po2profnow.max()>=thp):
                        didxdeep = np.argmax(po2profnow<=thp).values
                        if didxdeep>0:
                        # get rid of case in which you have a strangely tiny po2 pressure < thp
                        # at the very top of the profile (i.e., either start measuring in mid depth of
                        # OMZ or you have erroneously small po2 pressure at/near the surface)
                            interpf = spinterpolate.interp1d(
                                po2profnow.isel(depth=slice(didxdeep-1,didxdeep+1)),
                                po2profnow['depth'].isel(depth=slice(didxdeep-1,didxdeep+1)))
                            thd[itime,ilat,ilon] = interpf(thp)
    thd.to_netcdf(dpath + 'WOD18/thd_195501-201807_5deg.nc')
elif loadthd==1:
    ncfile = dpath + 'WOD18/thd_195501-201807_5deg.nc' 
    thd = xr.open_dataset(ncfile) # m
    thd = thd['thd']

In [17]:
# Compute thermocline depth (TCD)
loadtcd = 1; tcdtype = 'variable'; # choose 'variable','20degC'
if loadtcd==0:
    tcd = xr.full_like(sst,np.nan)
    tcd.name = 'tcd'; tcd.attrs['units'] = 'm';
    tcd.attrs['long_name'] = 'thermocline depth (' + tcdtype + 'isotherm method)'
    if tcdtype=='20degC':
        ttcnow = 20;
    elif tcdtype=='variable':
        # Calculate thermocline temperature 3D (x,y,time) map for variable isotherm method
        tmld = sst-0.8
        ttc = tmld-0.25*(tmld-temp.sel(depth=400))
    for ilon in range(tcd.lon.size):
        for ilat in range(tcd.lat.size):
            for itime in range(tcd.time.size):
                tempprofnow = temp.isel(lat=ilat,lon=ilon,time=itime)
                tempprofnow = tempprofnow[~np.isnan(tempprofnow)]
                if len(tempprofnow)>0: # get rid of case in which tempprofnow is all nans
                    if tcdtype=='variable':
                        ttcnow = ttc.isel(lat=ilat,lon=ilon,time=itime).values
                    if (tempprofnow.min()<=ttcnow) & (tempprofnow.max()>=ttcnow):
                        didxdeep = np.argmax(tempprofnow<=ttcnow).values
                        if didxdeep>0:
                        # get rid of case in which you have a strangely tiny temp pressure < ttcnow
                        # at the very top of the profile (i.e., either start measuring in mid depth of
                        # OMZ or you have erroneously small temp pressure at/near the surface)
                            interpf = spinterpolate.interp1d(
                                tempprofnow.isel(depth=slice(didxdeep-1,didxdeep+1)),
                                tempprofnow['depth'].isel(depth=slice(didxdeep-1,didxdeep+1)))
                            tcd[itime,ilat,ilon] = interpf(ttcnow)
    tcd.to_netcdf(dpath + 'WOD18/tcd_195501-201807_5deg.nc')
elif loadtcd==1:
    ncfile = dpath + 'WOD18/tcd_195501-201807_5deg.nc' 
    tcd = xr.open_dataset(ncfile) # m
    tcd = tcd['tcd']

# Load chl data

In [20]:
ncfile = dpath + 'chl/ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHLY_5degWCPFCgrid_GEO_PML_OCx-199709-201812-fv4.0.nc'
chl = xr.open_dataset(ncfile) # mg/m^3

#### Correct the time axis

In [21]:
# - Time goes from Sep 1997 - Dec 2018 --> 256 months
begdatechl = datetime.date(year = 1997, month = 9, day = 1)
enddatechl = datetime.date(year = 2018, month = 12, day = 1)
timemochl = pd.date_range(begdatechl, enddatechl, freq='MS')
chl['TIME'] = timemochl

#### Rename

In [22]:
logchl = np.log10(chl['CHLA'])
logchl = logchl.rename({'LON':'lon', 'LAT':'lat', 'TIME':'time'})
logchl.name = 'logchl'

# Load MLD data

In [23]:
ncfile = dpath + 'ecco/MXLDEPTH_1992_01-2017_12.WCPFC5deggrid.nc'
MLD = xr.open_dataset(ncfile) # m
mld = MLD['MLD']
mld = mld.where(mld!=0)

#### Correct the time axis

In [ ]:
# - Time goes from Jan 1992 - Dec 2017 --> 312 months
begdatemld = datetime.date(year = 1992, month = 1, day = 1)
enddatemld = datetime.date(year = 2017, month = 12, day = 1)
timemomld = pd.date_range(begdatemld, enddatemld, freq='MS')
mld['TIME'] = timemomld

#### Rename

In [25]:
mld = mld.rename({'LON':'lon', 'LAT':'lat', 'TIME':'time'})
mld.name = 'mld'

# Load SSHA data

In [205]:
combinedssha = xr.merge([xr.open_dataset(f) for f in glob(dpath + 'aviso/msla/*.nc')])
ssha = 100*combinedssha['SSHA'] # cm

#### Correct the time axis

In [207]:
# - Time goes from Jan 1993 - Dec 2018 --> 312 months
begdatessha = datetime.date(year = 1993, month = 1, day = 1)
enddatessha = datetime.date(year = 2018, month = 12, day = 1)
timemossha = pd.date_range(begdatessha, enddatessha, freq='MS')
ssha['TIME'] = timemossha

#### Rename

In [208]:
ssha = ssha.rename({'LON':'lon', 'LAT':'lat', 'TIME':'time'})
ssha.name = 'ssha'